In [41]:
import os
import cv2
import numpy as np

In [44]:
def enhance_image(image):
    height, width = image.shape[:2]
    left_half = image[:, :width // 2]

    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    image_clahe = cv2.cvtColor(left_half, cv2.COLOR_BGR2LAB)
    image_clahe[:, :, 0] = clahe.apply(image_clahe[:, :, 0])  
    image_clahe = cv2.cvtColor(image_clahe, cv2.COLOR_LAB2BGR)

    gamma = 1.3  
    look_up_table = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in np.arange(0, 256)]).astype("uint8")
    image_gamma_corrected = cv2.LUT(image_clahe, look_up_table)

    alpha = 1.2 
    beta = 15    
    image_dehazed = cv2.convertScaleAbs(image_gamma_corrected, alpha=alpha, beta=beta)

    denoised_image = cv2.fastNlMeansDenoisingColored(image_dehazed, None, h=10, templateWindowSize=7, searchWindowSize=21)

    denoised_image_rgb = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2RGB)

    return denoised_image_rgb


In [46]:
def detect_edges(image):
    height, width = image.shape[:2]
    left_half = image[:, :width // 2]

    gray_image = cv2.cvtColor(left_half, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, threshold1=50, threshold2=150)

    dilated_edges = cv2.dilate(edges, None, iterations=1)
    eroded_edges = cv2.erode(dilated_edges, None, iterations=1)

    lines = cv2.HoughLinesP(eroded_edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=10)
    line_image = np.zeros_like(left_half)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 2)

    contours, _ = cv2.findContours(eroded_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = left_half.copy()
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    combined_output = cv2.addWeighted(left_half, 0.7, line_image, 1, 0)
    combined_output = cv2.addWeighted(combined_output, 0.7, contour_image, 0.3, 0)

    final_image = Image.fromarray(cv2.cvtColor(combined_output, cv2.COLOR_BGR2RGB))

    return final_image

In [47]:
def reduce_noise(image):
    height, width = image.shape[:2]
    left_half = image[:, :width // 2]

    gaussian_blurred = cv2.GaussianBlur(left_half, (5, 5), 0)
    bilateral_filtered = cv2.bilateralFilter(gaussian_blurred, d=9, sigmaColor=75, sigmaSpace=75)
    denoised_image = cv2.fastNlMeansDenoisingColored(bilateral_filtered, None, h=10, templateWindowSize=7, searchWindowSize=21)

    sharpening_kernel = np.array([[-1, -1, -1],
                                [-1, 9, -1],
                                [-1, -1, -1]])
    sharpened_image = cv2.filter2D(denoised_image, -1, sharpening_kernel)
    final_image_rgb = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2RGB)

    return final_image_rgb


In [ ]:
import cv2
import os
import numpy as np
from PIL import Image  

def process_images(image_dir, output_dir):
    enhanced_dir = os.path.join(output_dir, "enhanced")
    edges_dir = os.path.join(output_dir, "edges")
    noise_reduced_dir = os.path.join(output_dir, "noise_reduced")
    
    os.makedirs(enhanced_dir, exist_ok=True)
    os.makedirs(edges_dir, exist_ok=True)
    os.makedirs(noise_reduced_dir, exist_ok=True)
    
    for img_file in os.listdir(image_dir):
        if img_file.endswith(".png") or img_file.endswith(".jpg"):
            image_path = os.path.join(image_dir, img_file)
            image = cv2.imread(image_path)
            
            enhanced_image = enhance_image(image)
            edges_image = detect_edges(image)
            noise_reduced_image = reduce_noise(image)
            
            if isinstance(edges_image, Image.Image):
                edges_image = np.array(edges_image)  

            
            cv2.imwrite(os.path.join(enhanced_dir, f"enhanced_{img_file}"), enhanced_image)
            cv2.imwrite(os.path.join(edges_dir, f"edges_{img_file}"), edges_image)
            cv2.imwrite(os.path.join(noise_reduced_dir, f"noise_reduced_{img_file}"), noise_reduced_image)

image_dir = "C:\\Users\\User\\OneDrive\\Documents\\Computer vision with DL\\image processing exam\\cityscapes-image-pairs\\cityscapes_data\\train"  
output_dir = "C:\\Users\\User\\OneDrive\\Documents\\Computer vision with DL\\image processing exam\\Output Image"  

process_images(image_dir, output_dir)
